In [ ]:
#| default_exp core

# fhswiftui.core

> API for helpers, layout components, UI components, and style modifiers.

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *

## Basics

In [ ]:
#| export
#| hide
def _def_colors():
    base = ["primary","secondary","accent","muted","card","popover"]
    return base+[f"{o}-foreground" for o in base]+["background","foreground","destructive","ring","input","border"]
    

In [ ]:
#| export
def IncludeColors(
    colors=None,    # list of tailwind colors
    append=True     # append or replace the default colors
):
    "Include additional colors that wil be used in the page"
    from itertools import product
    if not colors: colors = []
    pr = [f"border-{o}" for o in "lrtb"] + ["bg","text","border"]
    
    return Div(cls=f"hidden {' '.join(f'{p}-{c}' for p,c in product(pr,_def_colors() + colors if append else colors))}")

In [ ]:
#| export
# enable basecoat and tailwind; add useful default colors
bc_link = Link(rel='stylesheet', href='https://cdn.jsdelivr.net/npm/basecoat-css@latest/dist/basecoat.cdn.min.css')
tw_scr = Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')

fh_swiftui_hdrs = (bc_link,tw_scr,IncludeColors())


In [ ]:

fh_swiftui_hdrs

(link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/basecoat-css@latest/dist/basecoat.cdn.min.css'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'}),
 div((),{'class': 'hidden border-l-primary border-l-secondary border-l-accent border-l-muted border-l-card border-l-popover border-l-primary-foreground border-l-secondary-foreground border-l-accent-foreground border-l-muted-foreground border-l-card-foreground border-l-popover-foreground border-l-background border-l-foreground border-l-destructive border-l-ring border-l-input border-l-border border-r-primary border-r-secondary border-r-accent border-r-muted border-r-card border-r-popover border-r-primary-foreground border-r-secondary-foreground border-r-accent-foreground border-r-muted-foreground border-r-card-foreground border-r-popover-foreground border-r-background border-r-foreground border-r-destructive border-r-ring border-r-input border-r-border border-t-primary border-t-secondary bord

In [ ]:
#| export
def mk_previewer(app=None,hdrs=fh_swiftui_hdrs,cls=f'max-w-lg'):
    xcls = cls
    if not app: app=FastHTML(hdrs=hdrs)
    def p(*c, cls='', **kw):
        return HTMX(Div(cls=f'{xcls} {cls}')(*c), app=app, host=None, port=None, **kw)
    return p

# Based completely on: https://answerdotai.github.io/fhdaisy/core.html#mk_previewer

In [ ]:
#| export
p = mk_previewer()

In [ ]:
c = Button("hello", cls="btn")
p(c)

In [ ]:
c

```html
<button class="btn">hello</button>
```

## Style modifiers

In [ ]:
#| export
@patch
def append_classes(
    self:FT, # the tag
    *c       # list of classes
):
    "Append css classes to the FastTag element"
    self.attrs["class"] = " ".join(f"{self.attrs.get('class','')} {' '.join(c)}".split())
    return self

### Padding - space inside the component

In [ ]:
#| export
@patch
def padding(
    self:FT,     # The FastTag element to modify
    **kw         # Padding values: 'all', 'top', 'bottom', 'left', 'right' (Tailwind spacing scale: 0, 1, 2, 4, 8, px, or custom like '[5px]')
):
    "Add padding around the content within the element's bounds. Defaults to p-4 if no arguments provided."
    d = {"top":'t', "bottom":'b', "left":'l', "right":'r'}
    c = [f"p-{kw['all']}"] if "all" in kw else [f"p{d[k]}-{kw[k]}" for k in set(kw) & d.keys()]
    return self.append_classes(*c if c else ["p-4"])

#### Tests

In [ ]:
#| export
# Test with different padding options
Div("Default", cls="bg-blue-200").padding()

```html
<div class="bg-blue-200 p-4">Default</div>

```

In [ ]:
#| export
# Test with different padding options
p(Div("Default", cls="bg-blue-200").padding())

In [ ]:
#| export
Div("All sides", cls="bg-blue-200").padding(all=6)

```html
<div class="bg-blue-200 p-6">All sides</div>

```

In [ ]:
#| export
p(Div("All sides", cls="bg-blue-200").padding(all=6))

In [ ]:
#| export
Div("Top and bottom", cls="bg-blue-200").padding(top=2, bottom=8)

```html
<div class="bg-blue-200 pb-8 pt-2">Top and bottom</div>

```

In [ ]:
#| export
p(Div("Top and bottom", cls="bg-blue-200").padding(top=2, bottom=8))

### Margins - space outside the component

In [ ]:
#| export
@patch
def margin(self:FT,**kw):
    d = {"top":'t',"bottom":'b',"left":'l',"right":'r'}
    c = [f"m-{kw['all']}"] if "all" in kw else [f"m{d[k]}-{kw[k]}" for k in set(kw) & d.keys()]

    self.append_classes(*c if c else ["m-4"])

    return self

#### Tests

In [ ]:
#| export
# Default margin
p(Div("Default", cls="bg-blue-200").margin())

In [ ]:
#| export
# All sides
p(Div("All sides", cls="bg-blue-200").margin(all=6))

In [ ]:
#| export
# Top and bottom
p(Div("Top and bottom", cls="bg-blue-200").margin(top=2, bottom=8))

### Borders

In [ ]:
#| export
@patch
def border(
    self:FT,         # The FastTag element to modify
    width:int=None,  # Border width in Tailwind scale (1, 2, 4, 8) or None for default width
    color:str=None   # Tailwind border color without 'border-' prefix (e.g., 'primary', 'muted', 'gray-300', 'red-500')
):
    "Add a border around the element with optional width and color customization"
    c = ["border"]
    if width: c.append(f"border-{width}")
    if color: c.append(f"border-{color}")
    return self.append_classes(*c)

#### Tests

In [ ]:
#| export
p(Div("Test").border())

In [ ]:
#| export
p(Div("Test").border(width=2,color="blue-500"))

In [ ]:
#| export
p(Div("Test").border(color="red-300"))

In [ ]:
#| export
p(Div("Test").border().padding().margin())

#| export
### Corner radius

In [ ]:
#| export
@patch
def corner_radius(
    self:FT,        # The FastTag element to modify
    sz:str=None     # Tailwind size suffix
):   
    "Round the corners for this element"
    return self.append_classes(f"rounded-{sz}" if sz else "rounded")


#### Tests

In [ ]:
#| export
# Default rounded
p(Div("Default", cls="bg-blue-200 p-4").corner_radius())

In [ ]:
#| export
# Specific sizes
p(Div("Small", cls="bg-green-200 p-4").corner_radius("sm"))

In [ ]:
#| export
p(Div("Large", cls="bg-red-200 p-4").corner_radius("lg"))

In [ ]:
#| export
p(Div("Full", cls="bg-yellow-200 p-4").corner_radius("full"))

In [ ]:
#| export
p(Div("Chaining").padding().border().corner_radius("lg"))

### bg/fg - set background and foreground colors

In [ ]:
#| export
@patch
def bg(
    self:FT,        # The FastTag element to modify
    color:str       # color of background
): 
    "Set background color"
    return self.append_classes(f"bg-{color}")

In [ ]:
#| export
@patch
def fg(
    self:FT,        # The FastTag element to modify
    color:str       # color of foreground
): 
    "Set foreground color"
    return self.append_classes(f"text-{color}")

#### Tests

In [ ]:
#| export
# Background color
p(Div("Blue background").padding().bg("blue-500"))

In [ ]:
#| export
# Foreground/text color
p(Div("Red text").padding().fg("red-600"))

In [ ]:
#| export
# Combined
p(Div("Yellow on purple", cls="p-4").bg("purple-700").fg("yellow-300"))

In [ ]:
#| export
# With other modifiers
p(Div("Styled box", cls="p-4").bg("green-200").fg("green-900").corner_radius("lg").border(width=2, color="green-500"))

### shadow - set shadow

In [ ]:
#| export
@patch
def shadow(self:FT,**kw):
    "Add shadow to element"
    # Parameters:
    # c: shadow color (default 'rgba(0,0,0,0.1)')
    # sz: preset size ('sm','md','lg', etc)
    # x,y,b,s: offset x, offset y, blur, spread
    if "c" not in kw: kw["c"] = "rgba(0,0,0,0.1)"
    
    if len(kw) == 0: return self.append_classes("shadow")
    if "sz" in kw: return self.append_classes(f"shadow-{kw['sz']}")
    
    def enc(c,x=0,y=1,b=3,s=0,**k2): return f"shadow-[{x}px_{y}px_{b}px_{s}px_{c}]"
    return self.append_classes(enc(**kw))

#### Tests

In [ ]:
#| export
# Default shadow
p(Button("Default Shadow", cls="btn btn-primary").shadow())

In [ ]:
#| export
# Custom color shadow
p(Button("Blue Shadow", cls="btn").shadow(c="rgba(0,0,255,0.5)", x=2, y=2, blur=8))

In [ ]:
#| export
# Large offset shadow
p(Button("Offset Shadow", cls="btn btn-secondary").shadow(x=4, y=8, b=12))

In [ ]:
#| export
# Red shadow
p(Button("Red Shadow", cls="btn btn-accent").shadow(color="red", blur=10))

### opacity - set opacity

In [ ]:
#| export
@patch
def opacity(
    self:FT,        # The FastTag element to modify
    v:int=100       # Percent value of opacity as int
):
    "set opacity"
    return self.append_classes(f"opacity-{v}")

#### Tests

In [ ]:
#| export
p(Div("Full opacity (default)", cls="bg-blue-500 p-4 text-white").opacity())

In [ ]:
#| export
p(Div("75% opacity", cls="bg-red-500 p-4 text-white").opacity(75))

In [ ]:
#| export
p(Div("50% opacity", cls="bg-green-500 p-4 text-white").opacity(50))

In [ ]:
#| export
p(Div("25% opacity", cls="bg-purple-500 p-4 text-white").opacity(25))

In [ ]:
#| export
p(Div(
    Div("100", cls="bg-blue-500 p-4 text-white").opacity(100),
    Div("75", cls="bg-blue-500 p-4 text-white").opacity(75),
    Div("50", cls="bg-blue-500 p-4 text-white").opacity(50),
    Div("25", cls="bg-blue-500 p-4 text-white").opacity(25)
))

### frame - position this within a frame

In [ ]:
#| export
@patch
def frame(
    self:FT,            # The FastTag element to modify 
    w=None,             # height of frame
    h=None,             # width of frame
    halign="center",    # horizontal alignment (default: center)
    valign="center"     # vertical alignment (default: center)
):
    "position this element in a frame"
    h_map = {"leading": "justify-start", "center": "justify-center", "trailing": "justify-end"}
    v_map = {"top": "items-start", "center": "items-center", "bottom": "items-end"}
    
    c = ["flex", h_map[halign], v_map[valign]]
    if w: c.append(f"w-{w}")
    if h: c.append(f"h-{h}")
    
    return Div(self).append_classes(*c)

#| export
#### Tests

In [ ]:
#| export
# Basic frame with size
p(Div("Content", cls="bg-blue-200 p-2").frame(w=64, h=32))

In [ ]:
#| export
# Different alignments
p(Div("Top Left", cls="bg-red-200 p-2").frame(w=96, h=48, valign="top",halign="leading"))

In [ ]:
#| export
p(Div("Center", cls="bg-green-200 p-2").frame(w=96, h=48, halign="center"))

In [ ]:
#| export
p(Div("Bottom Right", cls="bg-yellow-200 p-2").frame(w=96, h=48, valign="bottom",halign="trailing"))

In [ ]:
#| export
# Just width or height
p(Div("Wide", cls="bg-purple-200 p-2").frame(w="full"))

In [ ]:
#| export
p(Div("Tall", cls="bg-pink-200 p-2").frame(h=64))

### font - set the default font for the tag

In [ ]:
#| export
@patch
def font(
    self:FT,        # The FastTag element to modify 
    sz=None,        # size
    wt=None         # weight
):
    "Set default font for this tag"
    c = []
    if sz: c.append(f"text-{sz}")
    if wt: c.append(f"font-{wt}")
    return self.append_classes(*c)

#### Tests

In [ ]:
#| export
# Different font sizes
p(Div("Small text").font(sz="sm"))

In [ ]:
#| export
p(Div("Base text").font(sz="base"))

In [ ]:
#| export
p(Div("Large text").font(sz="lg"))

In [ ]:
#| export
p(Div("Extra large text").font(sz="xl"))

In [ ]:
#| export
p(Div("2XL text").font(sz="2xl"))

In [ ]:
#| export
# Different font weights
p(Div("Light text").font(wt="light"))

In [ ]:
#| export
p(Div("Normal text").font(wt="normal"))

In [ ]:
#| export
p(Div("Bold text").font(wt="bold"))

In [ ]:
#| export
# Combined size and weight
p(Div("Large and bold").font(sz="2xl",wt="bold"))

In [ ]:
#| export
p(Div("Small and light").font(sz="sm",wt="light"))